# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/
https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/
https://edwarddonner.com/2025/01/23/ll

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 'blog/inference-providers-cohere',
 '/spaces',
 '/models',
 '/microsoft/bitnet-b1.58-2B-4T',
 '/HiDream-ai/HiDream-I1-Full',
 '/microsoft/MAI-DS-R1',
 '/Shakker-Labs/FLUX.1-dev-ControlNet-Union-Pro-2.0',
 '/THUDM/GLM-4-32B-0414',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/bytedance-research/UNO-FLUX',
 '/spaces/jamesliu1217/EasyControl_Ghibli',
 '/spaces/InstantX/InstantCharacter',
 '/spaces/Yuanshi/OminiControl_Art',
 '/spaces',
 '/datasets/openai/mrcr',
 '/datasets/zwhe99/DeepMath-103K',
 '/datasets/nvidia/OpenCodeReasoning',
 '/datasets/openai/graphwalks',
 '/datasets/nvidia/Llama-Nemotron-Post-Training-Dataset',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel

In [11]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'github page', 'url': 'https://github.com/huggingface'},
  {'type': 'linkedin page',
   'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'twitter page', 'url': 'https://twitter.com/huggingface'},
  {'type': 'discussion forum', 'url': 'https://discuss.huggingface.co'},
  {'type': 'status page', 'url': 'https://status.huggingface.co/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [12]:
Website("https://huggingface.co").get_contents()

'Webpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nNEW\nWelcome Cohere on the Hub 🔥\nWelcome Hyperbolic, Nebius AI Studio, and Novita on the Hub 🔥\nWelcome Fireworks.ai on the Hub 🎆\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nmicrosoft/bitnet-b1.58-2B-4T\nUpdated\n1 day ago\n•\n13k\n•\n561\nHiDream-ai/HiDream-I1-Full\nUpdated\n5 days ago\n•\n24.3k\n•\n648\nmicrosoft/MAI-DS-R1\nUpdated\n4 days ago\n•\n284\n•\n174\nShakker-Labs/FLUX.1-dev-ControlNet-Union-Pro-2.0\nUpdated\n1 day ago\n•\n10.1k\n•\n157\nTHUDM/GLM-4-32B-0414\nUpdated\n7 days ago\n•\n4.19k\n•\n145\nBrowse 1M+ models\nSpaces\nRunning\n4.91k\n4.91k\nDeepSite\n🐳\nGenerate any application with DeepSeek\nRunning\non\nZero\n476\n476\nU

In [13]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'community page', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

In [14]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [15]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}
Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
NEW
Welcome Cohere on the Hub 🔥
Welcome Hyperbolic, Nebius AI Studio, and Novita on the Hub 🔥
Welcome Fireworks.ai on the Hub 🎆
The AI community building the fut

In [16]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."


In [17]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [18]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


"You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nNEW\nWelcome Cohere on the Hub 🔥\nWelcome Hyperbolic, Nebius AI Studio, and Novita on the Hub 🔥\nWelcome Fireworks.ai on the Hub 🎆\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nmicrosoft/bitnet-b1.58-2B-4T\nUpdated\n1 day ago\n•\n13k\n•\n561\nHiDream-ai/HiDream-I1-Full\nUpdated\n5 days ago\n•\n24.3k\n•\n648\nmicrosoft/MAI-DS-R1\nUpdated\n4 days ago\n•\n284\n•\n174\nShakker-Labs/FLUX.1-dev-ControlNet-Union-Pro-2.0\nUpdated\n1 day ago\n•\n

In [29]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))
    return result


In [30]:
english_brochure = create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}, {'type': 'community discussion page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}


```markdown
# Welcome to Hugging Face! 🤗

### The AI Community Building the Future (And Maybe Your Next Favorite Robot)
At Hugging Face, we’re not just coding – we’re crafting the future with a splash of creativity and a hint of machine learning magic! Join a community where models and algorithms are made to hug it out!

### What We Offer
- **1 Million Models. Literally.** Need an AI model? We’ve got more than a million to choose from! That’s right – we’re practically a model UN, but instead of countries, we’ve got neural networks! 🤖
- **Datasets Galore!** Dive into 250,000+ datasets. It's like a buffet, but for data!
- **Spaces That Spark Joy!** Create, discover, and run applications all in one place. Let’s face it – a good space is better than a good sock drawer!

### Our Deals are Hard to Resist
- **Compute Solutions** as low as **$0.60/hour.** That's cheaper than your last coffee run!
- **Enterprise Solutions** starting at **$20/user/month.** Give your team the Playground of Awesomeness – secure and equipped with priority support!🎢

### Who Are Our Customers?
Over **50,000 organizations** can’t be wrong:
- Big shots like Apple, Google, and Microsoft are in the club. It's exclusive, but you’re invited! 
- We also cater to inspiring innovators like Ai2, Grammarly, and even a few mysterious users who prefer to remain anonymous. 🕵️‍♀️ 

### Culture at Hugging Face: Where Fun Meets Functionality
- **Team Spirit:** Collaboration is our middle name (well, not literally). We embrace open source like a fluffy puppy! Everyone's welcome to contribute.
- **Work-Life Balance:** We firmly believe that you should hit the quota without sacrificing your social life (or your ability to binge-watch!). So we keep it balanced!
- **Average Workday:** Starts with coffee, ends with shared laughs, and includes frequent model-hugging breaks. It’s a little unconventional but – isn’t that what AI is all about? 

### Join Our Team!
Looking for a career where you can make a mark in AI without losing your sanity? **Look no further!** 
- Developers, marketers, and dreamers (yes, we want you too!) are welcome.
- Great benefits, fun colleagues, and the chance to work on groundbreaking projects await you! 

### Ready for the Future?
Join the Hugging Face family and help us shape AI into something beautiful, friendly, and actually useful. Because remember, in a world full of cold algorithms, there's nothing like a hug! 🤗 

[Sign Up Now](https://huggingface.co) and let’s create magic together!
```


In [21]:
def system_prompt_translate(): 

    system_prompt = f"You are an assistant that will be provided a brochure in english about a company that might include details about culture, customers, careers and so on." 
    system_prompt += f"Your task is to translate this brochure to specified language."
    system_prompt += f"Provided brochure is written in Markdown. Please follow the same structure."
    

    return system_prompt



In [22]:
def user_prompt_translate(result_english, language = 'Finnish'):
    user_prompt = f"Please translate following brochure to {language}.\n\n Here is the brochure in English:\n\n{result_english}\n\n"
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [34]:
def translate_brochure(result_english, language = 'Finnish'):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt_translate()},
            {"role": "user", "content": user_prompt_translate(result_english, language)}
          ],
        stream=True,
    )
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [35]:
translate_brochure(english_brochure, 'Slovak')


# Vitajte v Hugging Face! 🤗

### AI komunita budujúca budúcnosť (a možno aj vášho ďalšieho obľúbeného robota)
V Hugging Face nekódujeme len tak – vytvárame budúcnosť s nádychom kreativity a štipkou čarovania strojového učenia! Pridajte sa k komunite, kde sa modely a algoritmy objímajú!

### Čo ponúkame
- **1 milión modelov. Doslovne.** Potrebujete AI model? Máme viac ako milión na výber! Presne tak – sme prakticky modelové OSN, ale namiesto krajín máme neurónové siete! 🤖
- **Dáta ako neobmedzené množstvo!** Ponorte sa do viac ako 250 000 datasetov. Je to ako bufet, ale pre dáta!
- **Priestory, ktoré prinášajú radosť!** Vytvárajte, objavujte a spúšťajte aplikácie všetko na jednom mieste. Poďme si povedať pravdu – dobrý priestor je lepší ako dobrý šuflík na ponožky!

### Naše ponuky sú ťažko odolné
- **Riešenia výpočtovej techniky** už od **0,60 USD/hod.** To je lacnejšie ako vaša posledná káva!
- **Podnikové riešenia** začínajú na **20 USD/užívateľ/mesiac.** Dajte svojmu tímu ihrisko úžasnosti – bezpečné a vybavené prioritnou podporou! 🎢

### Kto sú naši zákazníci?
Viac ako **50 000 organizácií** sa nemôže mýliť:
- Veľké mená ako Apple, Google a Microsoft sú v klube. Je to exkluzívne, ale ste pozvaní! 
- Obsluhujeme aj inšpirujúcich innovátorov ako Ai2, Grammarly, a dokonca aj niekoľko tajomných užívateľov, ktorí dávajú prednosť anonymite. 🕵️‍♀️ 

### Kultura v Hugging Face: Kde zábava stretáva funkčnosť
- **Tímový duch:** Spolupráca je naše druhé meno (no, nie doslovne). Prijímame open source ako mäkké šteniatko! Každý je vítaný prispieť.
- **Vyváženosť medzi prácou a životom:** Tvrdíme, že by ste mali dosiahnuť kvótu bez obetovania svojho spoločenského života (alebo schopnosti binge-watchovania!). Preto to udržiavame vyvážené!
- **Priemerný pracovný deň:** Začína kávou, končí zdieľaným smiechom a obsahuje časté prestávky na objímanie modelov. Je to trochu nekonvenčné, ale – nie je to to, o čom je AI?

### Pridajte sa k nášmu tímu!
Hľadáte kariéru, kde môžete zanechať stopu v AI bez straty zdravého rozumu? **Hľadajte ďalej!** 
- Programátori, marketéri a snílkovia (áno, chceme aj vás!) sú vítaní.
- Čakajú vás skvelé benefity, zábavní kolegovia a možnosť pracovať na prelomových projektoch! 

### Pripravte sa na budúcnosť?
Pridajte sa k rodine Hugging Face a pomôžte nám formovať AI na niečo krásne, priateľské a naozaj užitočné. Pretože si pamätajte, v svete plnom studených algoritmov nie je nič ako objatie! 🤗 

[Prihláste sa teraz](https://huggingface.co) a vytvorme spolu mágiu!


## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [46]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)
        
        
    # for chunk in stream: 
    #     print(chunk.choices[0].delta.content, end='')

In [ ]:
stream_brochure("HuggingFace", "https://huggingface.co")

In [47]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'docs page', 'url': 'https://huggingface.co/docs'}]}



# Hugging Face Brochure

## Overview
**Hugging Face** is at the forefront of building the future of artificial intelligence and machine learning. As a vibrant community hub, it hosts a rich ecosystem of models, datasets, and applications, making collaboration accessible for all levels of expertise in AI.

---

## Our Mission
*The AI community building the future.*  
Hugging Face is dedicated to fostering an inclusive and innovative environment where researchers, developers, and businesses can come together to create and improve machine learning technologies.

---

## What We Offer

### **Models**
Explore a collection of **1 million+ models** ranging from the latest innovations in language processing to state-of-the-art vision and audio AI models.
- Example Trends:
  - **microsoft/bitnet-b1.58-2B-4T**
  - **HiDream-ai/HiDream-I1-Full**
  
### **Datasets**
Access over **250k datasets** suitable for various ML tasks, encouraging the sharing and utilization of data to fuel further advancements in the industry.

### **Spaces**
Engage with the community through **400k+ applications** in spaces such as image generation, character customization, and artistic transformations.

### **Enterprise Solutions**
We offer advanced platforms designed with enterprise-grade security, access control, and dedicated support:
- Starting at **$20/user/month** with insights into priority support and custom solutions.

---

## Customers
Hugging Face proudly serves **more than 50,000 organizations** across diverse sectors including:
- *Meta*
- *Amazon*
- *Google*
- *Intel*
- *Microsoft*

---

## Company Culture
At Hugging Face, we believe in **collaboration, open source**, and continuous learning. Our culture is driven by a passion for AI and a commitment to making cutting-edge technology accessible to everyone. Employees work in a dynamic environment that encourages creativity, innovation, and teamwork.

---

## Career Opportunities
Join our thriving community! Hugging Face is always looking for talented individuals passionate about transforming the future of AI. Whether you are a software engineer, researcher, or a marketing professional, we welcome you to contribute to our mission.
- Explore current openings at: [Hugging Face Careers](https://huggingface.co/jobs)

---

## Join Us
Whether you're a researcher, a developer, an enterprise, or a job seeker, become one of us and make a mark on the future of AI.  
**Sign Up Today** and start exploring the possibilities!

[Learn More](https://huggingface.co)



<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>